# Optimización del Activo
Import de datos

In [2]:
library("readxl")
library("lubridate")
library('tidyr')
library('ggplot2')

Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union

Warning message:
"package 'tidyr' was built under R version 3.6.3"

In [3]:
search()

[1] ".GlobalEnv"        "package:ggplot2"   "package:tidyr"    
 [4] "package:lubridate" "package:readxl"    "jupyter:irkernel" 
 [7] "jupyter:irkernel"  "package:stats"     "package:graphics" 
[10] "package:grDevices" "package:utils"     "package:datasets" 
[13] "package:methods"   "Autoloads"         "package:base"

In [4]:
source("helpers.R")

In [9]:
path <- "../Taller 2/data/Datos Insumo (1).xlsx"

In [10]:
names <- excel_sheets(path)
names

ERROR: Error: `path` does not exist: '../Taller 2/data/Datos lnsumo (1).xlsx'
